<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/ML-Datasets/blob/master/Tf_2_0_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install tqdm

In [5]:
!wget --no-check-certificate \ https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

 https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip: Scheme missing.


<h3>Importing the required files

In [6]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Unzipping the data

In [7]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [8]:
zip_object = zipfile.ZipFile(file=dataset_path,mode='r')

In [9]:
zip_object.extractall("./")

In [10]:
zip_object.close()

### Setting up data path

In [11]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [29]:
training_dir = os.path.join(dataset_path_new,"train")
testing_dir = os.path.join(dataset_path_new,"validation")

## Building the image

### Loading MobileNetV2

In [13]:
image_shape = (128,128,3)

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=image_shape,include_top=False)

9412608/9406464 [==============================] - 0s 0us/step


In [15]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model and defining the custom head

In [16]:
base_model.trainable=False

In [17]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [18]:
# Using global average pooling
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [35]:
prediction_layer = tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

### Defining and compiling the model

In [36]:
model = tf.keras.models.Model(inputs=base_model.input, outputs = prediction_layer)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [37]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

### Preprocessing the data using ImageDataGenerator

In [38]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_test = ImageDataGenerator(rescale=1/255.0)

In [39]:
train_generator = data_gen_train.flow_from_directory(training_dir,target_size=(128,128),batch_size=128, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [40]:
test_generator = data_gen_test.flow_from_directory(testing_dir,target_size=(128,128),batch_size=128, class_mode='binary')

Found 1000 images belonging to 2 classes.


In [41]:
model.fit_generator(train_generator,epochs=10,validation_data=test_generator)

Epoch 1/10
16/16 [==============================] - 10s 633ms/step - loss: 0.7039 - accuracy: 0.5955 - val_loss: 0.6488 - val_accuracy: 0.6390
Epoch 2/10
16/16 [==============================] - 9s 573ms/step - loss: 0.6000 - accuracy: 0.6870 - val_loss: 0.5661 - val_accuracy: 0.7050
Epoch 3/10
16/16 [==============================] - 9s 569ms/step - loss: 0.5272 - accuracy: 0.7465 - val_loss: 0.4996 - val_accuracy: 0.7590
Epoch 4/10
16/16 [==============================] - 9s 573ms/step - loss: 0.4648 - accuracy: 0.7980 - val_loss: 0.4433 - val_accuracy: 0.8100
Epoch 5/10
16/16 [==============================] - 9s 568ms/step - loss: 0.4132 - accuracy: 0.8325 - val_loss: 0.3946 - val_accuracy: 0.8400
Epoch 6/10
16/16 [==============================] - 9s 566ms/step - loss: 0.3694 - accuracy: 0.8635 - val_loss: 0.3539 - val_accuracy: 0.8620
Epoch 7/10
16/16 [==============================] - 9s 580ms/step - loss: 0.3320 - accuracy: 0.8850 - val_loss: 0.3196 - val_accuracy: 0.8790
Epoch

<h3>Evaluating the dataset

In [44]:
valid_loss, valid_accuracy = model.evaluate_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [45]:
print("Accuracy after Transfer Learning: {}".format(valid_accuracy))

Accuracy after Transfer Learning: 0.9110000133514404


### Performing Fine Tuning

In [46]:
base_model.trainable = True

In [48]:
print("Layers of the base model: {}".format(len(base_model.layers)))

Layers of the base model: 155


In [51]:
fine_tune_at=100

In [52]:
# We are freezing all layers from 100-155
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False

In [53]:
# Compiling the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics=['accuracy'])

In [54]:
model.fit(train_generator,epochs=10,validation_data=test_generator)

Epoch 1/10
16/16 [==============================] - 10s 650ms/step - loss: 0.1706 - accuracy: 0.9300 - val_loss: 0.0794 - val_accuracy: 0.9680
Epoch 2/10
16/16 [==============================] - 10s 597ms/step - loss: 0.0230 - accuracy: 0.9985 - val_loss: 0.0829 - val_accuracy: 0.9710
Epoch 3/10
16/16 [==============================] - 9s 592ms/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.0883 - val_accuracy: 0.9700
Epoch 4/10
16/16 [==============================] - 9s 594ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0996 - val_accuracy: 0.9690
Epoch 5/10
16/16 [==============================] - 10s 597ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.1233 - val_accuracy: 0.9680
Epoch 6/10
16/16 [==============================] - 9s 586ms/step - loss: 6.3209e-04 - accuracy: 1.0000 - val_loss: 0.1042 - val_accuracy: 0.9740
Epoch 7/10
16/16 [==============================] - 9s 588ms/step - loss: 3.6723e-04 - accuracy: 1.0000 - val_loss: 0.1204 - val_accuracy: 0.

In [55]:
valid_loss, valid_accuracy= model.evaluate(test_generator)

8/8 [==============================] - 3s 340ms/step - loss: 0.1899 - accuracy: 0.9630


In [56]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9629999995231628
